In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence
from sklearn.model_selection import train_test_split
import torch.nn.functional as F

In [2]:
df = pd.read_csv('C:/Users/JenMing/Desktop/MBTI/LSTM/mbti_to_LSTM_DF.csv')
df.head()

,type,posts
0,INFJ,"['INFP', 'INFP', 'INFJ', 'ENFP', 'ISTP', 'INTP..."
1,ENTP,"['INTJ', 'INTP', 'ENFP', 'INTJ', 'INTP', 'INTP..."
2,INTP,"['INTJ', 'INFP', 'INFP', 'INTP', 'INTP', 'INTJ..."
3,INTJ,"['INTJ', 'ISFJ', 'INFP', 'INTP', 'INTP', 'INTP..."
4,ENTJ,"['ENTJ', 'INTP', 'ENFP', 'INTP', 'ENTJ', 'INTJ..."


In [21]:
print(df['posts'][12])
print(df['type'][12])

['ISTP', 'ISTP', 'INFP', 'INFP', 'INFJ', 'INFJ', 'INFP', 'INFP', 'INTP', 'INTJ', 'INFJ', 'INFP', 'ENFP', 'INFP', 'INTJ', 'ENFP', 'ENTP', 'ENFP', 'ENFP', 'INTP', 'INFP', 'ENFP', 'INFJ', 'ENFP', 'ISFJ', 'INFP', 'INTJ', 'ISTP', 'ENFP', 'INTJ', 'INTJ', 'ENFP', 'ENFP', 'ENFP', 'ENFP', 'ENFP', 'ENFP', 'ISFP', 'ENFP', 'ENFP', 'INFJ', 'INFP', 'INFP', 'ENTP', 'INFP', 'INTP', 'ENFP', 'INTJ', 'ENTP']
INFJ


In [4]:
# 編碼轉換
personality_mapping = {'INFJ': 0,
                        'ENTP': 1,
                        'INTP': 2,
                        'INTJ': 3,
                        'ENTJ': 4,
                        'ENFJ': 5,
                        'INFP': 6,
                        'ENFP': 7,
                        'ISFP': 8,
                        'ISTP': 9,
                        'ISFJ': 10,
                        'ISTJ': 11,
                        'ESTP': 12,
                        'ESFP': 13,
                        'ESTJ': 14,
                        'ESFJ': 15 }

In [5]:
# 資料載入和轉換
encoded_data = []

chars_to_remove = "][' "    

for index, row in df.iterrows():
    mbti_counts = {0:0,1:0,2:0,3:0,4:0,5:0,6:0,7:0,8:0,9:0,10:0,11:0,12:0,13:0,14:0,15:0}
    mbti_per_count = []
    dialogues = row["posts"] #字串
    target_personality = row["type"]
    for char in chars_to_remove:
        dialogues = dialogues.replace(char, "")
    
    dialogues_list = dialogues.split(',')
    
    
    dialogue_ids = [personality_mapping[personality] for personality in dialogues_list]
    
    for personality_id in dialogue_ids:
        mbti_counts[personality_id] += 1
    
    for i in range(len(personality_mapping)):
        mbti_per_count.append(round(mbti_counts[i]/len(dialogue_ids), 2))
   
    print(mbti_per_count)
    tmp_mbti_per_count = mbti_per_count
    for index, personality_num in enumerate(mbti_per_count):
        if personality_num < (1/len(personality_mapping)):
            mbti_per_count[index] = 0.0
            
    print(mbti_per_count)
    
    # 计算所有非零值的总和
    non_zero_sum = np.sum([x for x in mbti_per_count if x != 0])
    
    #如果都是0
    if non_zero_sum == 0: 
        mbti_per_count = tmp_mbti_per_count
    else:  # 重新分配其他非零值，使它们的总和为1
        # 重新分配其他非零值，使它们的总和为1
        mbti_per_count = [round(x / non_zero_sum, 2) if x != 0 else 0 for x in mbti_per_count]
        
    print(mbti_per_count)
    
    
    
    target_personality_id = personality_mapping[target_personality]
    
    encoded_data.append((mbti_per_count, target_personality_id))

[0.14, 0.04, 0.18, 0.11, 0.0, 0.0, 0.21, 0.14, 0.0, 0.18, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.14, 0.0, 0.18, 0.11, 0.0, 0.0, 0.21, 0.14, 0.0, 0.18, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.15, 0, 0.19, 0.11, 0, 0, 0.22, 0.15, 0, 0.19, 0, 0, 0, 0, 0, 0]
[0.02, 0.02, 0.16, 0.16, 0.02, 0.0, 0.07, 0.33, 0.05, 0.09, 0.02, 0.05, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.16, 0.16, 0.0, 0.0, 0.07, 0.33, 0.0, 0.09, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0, 0, 0.2, 0.2, 0, 0, 0.09, 0.41, 0, 0.11, 0, 0, 0, 0, 0, 0]
[0.15, 0.03, 0.33, 0.15, 0.0, 0.0, 0.26, 0.08, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.15, 0.0, 0.33, 0.15, 0.0, 0.0, 0.26, 0.08, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.15, 0, 0.34, 0.15, 0, 0, 0.27, 0.08, 0, 0, 0, 0, 0, 0, 0, 0]
[0.06, 0.06, 0.33, 0.1, 0.02, 0.0, 0.1, 0.1, 0.0, 0.17, 0.02, 0.02, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.33, 0.1, 0.0, 0.0, 0.1, 0.1, 0.0, 0.17, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0, 0, 0.41, 0.12, 0, 0, 0.12, 0.12, 0, 0.21, 0, 0, 0, 0, 0, 0]
[0.14, 0.11, 0.14, 0.27, 0.07, 0.0, 0.09, 0.09

[0.15, 0, 0.24, 0.15, 0, 0, 0.15, 0.31, 0, 0, 0, 0, 0, 0, 0, 0]
[0.1, 0.04, 0.28, 0.18, 0.02, 0.0, 0.14, 0.16, 0.0, 0.08, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.1, 0.0, 0.28, 0.18, 0.0, 0.0, 0.14, 0.16, 0.0, 0.08, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.11, 0, 0.3, 0.19, 0, 0, 0.15, 0.17, 0, 0.09, 0, 0, 0, 0, 0, 0]
[0.04, 0.12, 0.17, 0.29, 0.0, 0.0, 0.08, 0.12, 0.0, 0.08, 0.08, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.12, 0.17, 0.29, 0.0, 0.0, 0.08, 0.12, 0.0, 0.08, 0.08, 0.0, 0.0, 0.0, 0.0, 0.0]
[0, 0.13, 0.18, 0.31, 0, 0, 0.09, 0.13, 0, 0.09, 0.09, 0, 0, 0, 0, 0]
[0.18, 0.1, 0.2, 0.16, 0.02, 0.0, 0.08, 0.14, 0.0, 0.04, 0.04, 0.02, 0.0, 0.0, 0.0, 0.0]
[0.18, 0.1, 0.2, 0.16, 0.0, 0.0, 0.08, 0.14, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.21, 0.12, 0.23, 0.19, 0, 0, 0.09, 0.16, 0, 0, 0, 0, 0, 0, 0, 0]
[0.12, 0.0, 0.22, 0.04, 0.02, 0.0, 0.3, 0.2, 0.02, 0.04, 0.02, 0.02, 0.0, 0.0, 0.0, 0.0]
[0.12, 0.0, 0.22, 0.0, 0.0, 0.0, 0.3, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.14, 0, 0.26, 0, 0, 0, 0.36, 0.24,

[0.18, 0.02, 0.02, 0.04, 0.0, 0.0, 0.33, 0.27, 0.02, 0.04, 0.06, 0.02, 0.0, 0.0, 0.0, 0.0]
[0.18, 0.0, 0.0, 0.0, 0.0, 0.0, 0.33, 0.27, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.23, 0, 0, 0, 0, 0, 0.42, 0.35, 0, 0, 0, 0, 0, 0, 0, 0]
[0.2, 0.09, 0.27, 0.18, 0.0, 0.0, 0.09, 0.09, 0.0, 0.09, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.2, 0.09, 0.27, 0.18, 0.0, 0.0, 0.09, 0.09, 0.0, 0.09, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.2, 0.09, 0.27, 0.18, 0, 0, 0.09, 0.09, 0, 0.09, 0, 0, 0, 0, 0, 0]
[0.1, 0.0, 0.36, 0.14, 0.0, 0.0, 0.18, 0.12, 0.0, 0.04, 0.0, 0.06, 0.0, 0.0, 0.0, 0.0]
[0.1, 0.0, 0.36, 0.14, 0.0, 0.0, 0.18, 0.12, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.11, 0, 0.4, 0.16, 0, 0, 0.2, 0.13, 0, 0, 0, 0, 0, 0, 0, 0]
[0.06, 0.06, 0.3, 0.12, 0.0, 0.0, 0.14, 0.18, 0.02, 0.08, 0.0, 0.04, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.3, 0.12, 0.0, 0.0, 0.14, 0.18, 0.0, 0.08, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0, 0, 0.37, 0.15, 0, 0, 0.17, 0.22, 0, 0.1, 0, 0, 0, 0, 0, 0]
[0.2, 0.02, 0.06, 0.08, 0.02, 0.02, 0.22, 0.3, 0.02, 

[0.27, 0, 0, 0.27, 0, 0, 0.27, 0.2, 0, 0, 0, 0, 0, 0, 0, 0]
[0.17, 0.04, 0.11, 0.04, 0.0, 0.04, 0.35, 0.15, 0.0, 0.09, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.17, 0.0, 0.11, 0.0, 0.0, 0.0, 0.35, 0.15, 0.0, 0.09, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.2, 0, 0.13, 0, 0, 0, 0.4, 0.17, 0, 0.1, 0, 0, 0, 0, 0, 0]
[0.19, 0.02, 0.09, 0.02, 0.0, 0.0, 0.19, 0.44, 0.02, 0.02, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.19, 0.0, 0.09, 0.0, 0.0, 0.0, 0.19, 0.44, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.21, 0, 0.1, 0, 0, 0, 0.21, 0.48, 0, 0, 0, 0, 0, 0, 0, 0]
[0.17, 0.09, 0.11, 0.09, 0.0, 0.02, 0.15, 0.23, 0.02, 0.09, 0.0, 0.04, 0.0, 0.0, 0.0, 0.0]
[0.17, 0.09, 0.11, 0.09, 0.0, 0.0, 0.15, 0.23, 0.0, 0.09, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.18, 0.1, 0.12, 0.1, 0, 0, 0.16, 0.25, 0, 0.1, 0, 0, 0, 0, 0, 0]
[0.09, 0.02, 0.17, 0.0, 0.0, 0.0, 0.36, 0.32, 0.0, 0.02, 0.0, 0.02, 0.0, 0.0, 0.0, 0.0]
[0.09, 0.0, 0.17, 0.0, 0.0, 0.0, 0.36, 0.32, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.1, 0, 0.18, 0, 0, 0, 0.38, 0.34, 0, 0, 0, 0, 0, 

'''
# 資料載入和轉換
encoded_data = []

chars_to_remove = "][' "    
total_count_del = 0

for index, row in df.iterrows():
    count_del = 0
    mbti_counts = {0:0,1:0,2:0,3:0,4:0,5:0,6:0,7:0,8:0,9:0,10:0,11:0,12:0,13:0,14:0,15:0}
    mbti_per_count = []
    dialogues = row["posts"] #字串
    target_personality = row["type"]
    for char in chars_to_remove:
        dialogues = dialogues.replace(char, "")
    
    dialogues_list = dialogues.split(',')
    new_dialogues_list = []
    #print(dialogues_list)
    for n in range(len(dialogues_list)):
        counter = 0
        for m in range(4):
            if dialogues_list[n][m] == target_personality[m]:
                counter += 1
        if counter >= 3:
            new_dialogues_list.append(dialogues_list[n])
        else:
            count_del += 1
    #print(new_dialogues_list)
    if len(new_dialogues_list) == 0:
        dialogue_ids = [personality_mapping[personality] for personality in dialogues_list]
        count_del = 0
    else:
        dialogue_ids = [personality_mapping[personality] for personality in new_dialogues_list]
    
    
    #print(dialogue_ids)
    
    for personality_id in dialogue_ids:
        mbti_counts[personality_id] += 1
    
    for i in range(len(personality_mapping)):
        mbti_per_count.append(round(mbti_counts[i]/len(dialogue_ids), 2))
    
    target_personality_id = personality_mapping[target_personality]
    
    encoded_data.append((mbti_per_count, target_personality_id))
    
    total_count_del += count_del
    
print(total_count_del)
'''

In [6]:
# 填充序列並轉換為張量
input_data = torch.tensor([feature for feature, _ in encoded_data], dtype=torch.float32)
target_personality = torch.tensor([target for _, target in encoded_data], dtype=torch.int64)  # 使用int64类型，因为它是类别标签

In [7]:
# 資料集切分為訓練集和驗證集
train_dialogues, val_dialogues, train_target, val_target = train_test_split(input_data, target_personality, test_size=0.15, random_state=42)

In [8]:
print(val_dialogues[0])
print(val_target[0])

tensor([0.0900, 0.0000, 0.1600, 0.0000, 0.0000, 0.0000, 0.4300, 0.3200, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000])
tensor(8)


In [9]:
# 创建反向映射
reverse_personality_mapping = {v: k for k, v in personality_mapping.items()}

In [10]:
similar_tag = 0 #控制多少個指標以下要捨棄 (1~4)
total_del_count_times = 0
for index, personality_num in enumerate(train_target):
    del_count_times = 0
    # 使用反向映射将数字转换为对应的个性
    personality = reverse_personality_mapping.get(int(personality_num), "Unknown")
    tmp_dialogues = train_dialogues[index]
    for n in range(len(personality_mapping)):
        counter = 0
        for m in range(4):
            if reverse_personality_mapping[n][m] == personality[m]:
                counter += 1
        if counter < similar_tag:
            train_dialogues[index][n] = 0.0
            del_count_times += 1
            
    # 计算所有非零值的总和
    non_zero_sum = torch.sum(train_dialogues[index])

    #如果都是0
    if non_zero_sum == 0: 
        train_dialogues[index] = tmp_dialogues
    else:  # 重新分配其他非零值，使它们的总和为1
        train_dialogues[index] = train_dialogues[index] / non_zero_sum
        total_del_count_times += del_count_times

print(total_del_count_times)

0


In [11]:
from sklearn import svm

# 创建一个SVM分类器
svm_classifier = svm.SVC(kernel='rbf', C=0.5)

# 使用训练数据训练SVM模型
svm_classifier.fit(train_dialogues, train_target)

SVC(C=0.5)

In [12]:
# 使用训练好的SVM模型进行预测
predicted_labels = svm_classifier.predict(val_dialogues)


In [13]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# 计算准确度
accuracy = accuracy_score(val_target, predicted_labels)

# 计算精确度
precision = precision_score(val_target, predicted_labels, average='weighted')

# 计算召回率
recall = recall_score(val_target, predicted_labels, average='weighted')

# 计算F1分数
f1 = f1_score(val_target, predicted_labels, average='weighted')

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

Accuracy: 0.43
Precision: 0.37
Recall: 0.43
F1 Score: 0.39


C:\Users\JenMing\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [14]:
with open("C:/Users/JenMing/Desktop/MBTI/SVM/note.txt", "w") as f:
    dimension_counts = {'E/I': 0,
                        'S/N': 0,
                        'T/F': 0,
                        'J/P': 0}
    correct_predictions = 0

    item_count = len(predicted_labels)
    
    for n in range(len(predicted_labels)):
        for personality, value in personality_mapping.items():
            if value == predicted_labels[n]:
                mbti_labels_pre = personality
                break
        for personality, value in personality_mapping.items():
            if value == val_target[n]:
                mbti_labels_tru = personality 
                break

        if mbti_labels_pre == mbti_labels_tru:
            correct_predictions += 1

        for n in range(4):
            if n == 0:
                if mbti_labels_pre[n] == mbti_labels_tru[n]:
                    dimension_counts['E/I'] += 1
            elif n == 1:
                if mbti_labels_pre[n] == mbti_labels_tru[n]:
                    dimension_counts['S/N'] += 1
            elif n == 2:
                if mbti_labels_pre[n] == mbti_labels_tru[n]:
                    dimension_counts['T/F'] += 1
            elif n == 3:
                if mbti_labels_pre[n] == mbti_labels_tru[n]:
                    dimension_counts['J/P'] += 1


    accuracy = correct_predictions / len(predicted_labels) 

    f.write(f"del_count: {total_del_count_times}\n")
    f.write(f"Accuracy: {accuracy:.2f}\n")
    f.write(f"Precision: {precision:.2f}\n")
    f.write(f"Recall: {recall:.2f}\n")
    f.write(f"F1 Score: {f1:.2f}\n\n")
    
    print(f"Validation Accuracy: {accuracy*100:.4f}%")
    f.write(f"Validation Accuracy: {accuracy*100:.4f}%\n")
    
    EI_counts = dimension_counts['E/I']
    SN_counts = dimension_counts['S/N']
    TF_counts = dimension_counts['T/F']
    JP_counts = dimension_counts['J/P']
    print(f'E.I: {EI_counts}/{item_count} ')
    print('Accuracy: '+ str(EI_counts/item_count)+'\n')
    print(f'S.N: {SN_counts}/{item_count} ')
    print('Accuracy: '+ str(SN_counts/item_count)+'\n')
    print(f'T.F: {TF_counts}/{item_count} ')
    print('Accuracy: '+ str(TF_counts/item_count)+'\n')
    print(f'J.P: {JP_counts}/{item_count} ')
    print('Accuracy: '+ str(JP_counts/item_count)+'\n')
    
    f.write(f'E.I: {EI_counts}/{item_count} ')
    f.write('Accuracy: '+ str(EI_counts/item_count)+'\n')
    f.write(f'S.N: {SN_counts}/{item_count} ')
    f.write('Accuracy: '+ str(SN_counts/item_count)+'\n')
    f.write(f'T.F: {TF_counts}/{item_count} ')
    f.write('Accuracy: '+ str(TF_counts/item_count)+'\n')
    f.write(f'J.P: {JP_counts}/{item_count} ')
    f.write('Accuracy: '+ str(JP_counts/item_count)+'\n')

Validation Accuracy: 43.1644%
E.I: 1035/1302 
Accuracy: 0.7949308755760369

S.N: 1146/1302 
Accuracy: 0.880184331797235

T.F: 1028/1302 
Accuracy: 0.7895545314900153

J.P: 903/1302 
Accuracy: 0.6935483870967742



In [15]:
import joblib  # 或者可以使用pickle

# 保存SVM模型
model_filename = 'C:/Users/JenMing/Desktop/MBTI/SVM/svm_model.pkl'
joblib.dump(svm_classifier, model_filename)

['C:/Users/JenMing/Desktop/MBTI/SVM/svm_model.pkl']

In [16]:
# 加载SVM模型
#loaded_svm_model = joblib.load(model_filename)

# 使用加载的模型进行预测
#loaded_predicted_labels = loaded_svm_model.predict(val_dialogues)